# `pandas` - Reshaping Data and Pivots tables

__Contents__
1. Setup
1. Reshaping data by pivoting/melt/stacking and unstacking
1. Pivot tables

##Reference
Related/useful documentation:
- http://pandas.pydata.org/pandas-docs/stable/index.html
- https://pandas.pydata.org/pandas-docs/stable/dsintro.html
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html

## 1. Setup

Load libraries

In [6]:
import pandas  as pd
import numpy   as np
(pd.__version__,
 np.__version__
)

Out[ 61 ]: ('0.19.2', '1.11.1')

Load a DataFrame from the `imports-85.csv` CSV file. Set the column names.

In [8]:
column_names = ['symboling', 'normalized-losses', 'make', 'fuel-type',
                'aspiration', 'num-of-doors', 'body-style', 'drive-wheels',
                'engine-location', 'wheel-base', 'length', 'width',
                'height', 'curb-weight', 'engine-type', 'num-of-cylinders',
                'engine-size', 'fuel-system', 'bore', 'stroke',
                'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg',
                'highway-mpg', 'price']
import_df = pd.read_csv('/dbfs/mnt/datalab-datasets/file-samples/imports-85.csv',
                        names=[string.replace('-','_') for string in column_names],
                        na_values=['?']
                       )

Display basic information about each column of the DataFrame.

In [10]:
import_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
symboling 205 non-null int64
normalized_losses 164 non-null float64
make 205 non-null object
fuel_type 205 non-null object
aspiration 205 non-null object
num_of_doors 203 non-null object
body_style 205 non-null object
drive_wheels 205 non-null object
engine_location 205 non-null object
wheel_base 205 non-null float64
length 205 non-null float64
width 205 non-null float64
height 205 non-null float64
curb_weight 205 non-null int64
engine_type 205 non-null object
num_of_cylinders 205 non-null object
engine_size 205 non-null int64
fuel_system 205 non-null object
bore 201 non-null float64
stroke 201 non-null float64
compression_ratio 205 non-null float64
horsepower 203 non-null float64
peak_rpm 203 non-null float64
city_mpg 205 non-null int64
highway_mpg 205 non-null int64
price 201 non-null float64
dtypes: float64(11), int64(5), object(10)
memory usage: 41.7+ KB

Create another sample DataFrame `df` and display the first 5 rows.

In [12]:
import datetime
np.random.seed(0)
df = pd.DataFrame({'item': ['A']*6 + ['B']*6 + ['C']*6 + ['D']*6,
                    'quantity': np.random.randint(1000,size=24),
                    'value': np.random.randn(24),
                    'date': [datetime.datetime(2013, i, 1) for i in range(1, 7)]*4
                   })
df.head()

Out[ 64 ]: 
 date item quantity value
0 2013-01-01 A 684 0.761038
1 2013-02-01 A 559 0.121675
2 2013-03-01 A 629 0.443863
3 2013-04-01 A 192 0.333674
4 2013-05-01 A 835 1.494079

In [13]:
import datetime
np.random.seed(1)
icecream_sales = pd.DataFrame({'flavor': ['Chocolate']*6 + ['Vanilla']*6 + ['Cookie Dough']*6 + ['Green Tea']*6,
                    'quantity': np.random.randint(10,size=24)+1,
                    'profit': 1.5*np.random.random_integers(5,size=24),
                    'date': [datetime.datetime(2018, 4, i) for i in range(1, 7)]*4
                   })
icecream_sales.head()

Out[ 65 ]: 
 date flavor profit quantity
0 2018-04-01 Chocolate 3.0 6
1 2018-04-02 Chocolate 3.0 9
2 2018-04-03 Chocolate 3.0 10
3 2018-04-04 Chocolate 3.0 6
4 2018-04-05 Chocolate 1.5 1

##2. Reshaping data

### `pivot()` method

The `pivot()` method of a DataFrame object is used to create a new derived table out of the given one. `pivot()` method takes 3 arguements: 
- `index` 
- `columns` 
- `values`

Notice that we cannot aggregate using `pivot()` method and if either rows or columns are not unique, this method will fail.

Use `pivot` method to reshape the data into a time series format.

In [18]:
pivot_df = icecream_sales.pivot(index='date', columns='flavor', values='quantity')
pivot_df

Out[ 66 ]: 
flavor Chocolate Cookie Dough Green Tea Vanilla
date 
2018-04-01 6 6 8 2
2018-04-02 9 3 10 8
2018-04-03 10 5 2 7
2018-04-04 6 3 8 10
2018-04-05 1 5 1 3
2018-04-06 1 8 7 5

Pivoting by multiple columns. Notice that in `pivot` method, if not specify the `values` parameter, all remaining columns will be used and the result will have hierarchically indexed columns.

In [20]:
icecream_sales.pivot(index='date',columns='flavor')

Out[ 67 ]: 
 profit quantity \
flavor Chocolate Cookie Dough Green Tea Vanilla Chocolate Cookie Dough 
date 
2018-04-01 3.0 1.5 1.5 3.0 6 6 
2018-04-02 3.0 6.0 3.0 1.5 9 3 
2018-04-03 3.0 3.0 6.0 1.5 10 5 
2018-04-04 3.0 3.0 7.5 6.0 6 3 
2018-04-05 1.5 6.0 4.5 4.5 1 5 
2018-04-06 7.5 7.5 7.5 3.0 1 8 

 
flavor Green Tea Vanilla 
date 
2018-04-01 8 2 
2018-04-02 10 8 
2018-04-03 2 7 
2018-04-04 8 10 
2018-04-05 1 3 
2018-04-06 7 5

##`melt()` function

The `melt()` function and the `melt()` method of a DataFrame are useful to transform a DataFrame from wide to long format.

Create numeric index in the `pivot_df` DataFrame.

In [24]:
pivot_df.reset_index(inplace = True)
pivot_df

Out[ 68 ]: 
flavor date Chocolate Cookie Dough Green Tea Vanilla
0 2018-04-01 6 6 8 2
1 2018-04-02 9 3 10 8
2 2018-04-03 10 5 2 7
3 2018-04-04 6 3 8 10
4 2018-04-05 1 5 1 3
5 2018-04-06 1 8 7 5

Use the `melt()` function to reshape the `pivot_df` DataFrame. The `melt()` function in the below code cell takes 
- `pivot_df`: the name of dataframe to reshape
- `id_vars` : the column(s) to use as identifier variable(s)
- `var_name`: name to use for the ‘variable’ column
- `value_name`: name to use for the ‘value’ column.

In [26]:
pd.melt(pivot_df,id_vars=['date'],var_name='flavor',value_name='sales')

Out[ 69 ]: 
 date flavor sales
0 2018-04-01 Chocolate 6
1 2018-04-02 Chocolate 9
2 2018-04-03 Chocolate 10
3 2018-04-04 Chocolate 6
4 2018-04-05 Chocolate 1
5 2018-04-06 Chocolate 1
6 2018-04-01 Cookie Dough 6
7 2018-04-02 Cookie Dough 3
8 2018-04-03 Cookie Dough 5
9 2018-04-04 Cookie Dough 3
10 2018-04-05 Cookie Dough 5
11 2018-04-06 Cookie Dough 8
12 2018-04-01 Green Tea 8
13 2018-04-02 Green Tea 10
14 2018-04-03 Green Tea 2
15 2018-04-04 Green Tea 8
16 2018-04-05 Green Tea 1
17 2018-04-06 Green Tea 7
18 2018-04-01 Vanilla 2
19 2018-04-02 Vanilla 8
20 2018-04-03 Vanilla 7
21 2018-04-04 Vanilla 10
22 2018-04-05 Vanilla 3
23 2018-04-06 Vanilla 5

### `stack()` & `unstack()` method

The `stack` and `unstack` methods are designed to work together with MultiIndex objects. Create a MultiIndex object `df_multi` from the `icecream_sales` DataFrame by setting multiple indexes and sorting them.

In [29]:
df_multi = icecream_sales.set_index(['date','flavor']).sort_index()
df_multi.head(8)

Out[ 70 ]: 
 profit quantity
date flavor 
2018-04-01 Chocolate 3.0 6
 Cookie Dough 1.5 6
 Green Tea 1.5 8
 Vanilla 3.0 2
2018-04-02 Chocolate 3.0 9
 Cookie Dough 6.0 3
 Green Tea 3.0 10
 Vanilla 1.5 8

The `stack()` method “compresses”(stack) a level in the `df_multi`’s columns. The stacked level becomes the new lowest level in a MultiIndex on the columns:

In [31]:
df_multi.stack()\
        .rename_axis(['date','flavor','type'],axis=0)\
        .head(16)

Out[ 71 ]: 
date flavor type 
2018-04-01 Chocolate profit 3.0
 quantity 6.0
 Cookie Dough profit 1.5
 quantity 6.0
 Green Tea profit 1.5
 quantity 8.0
 Vanilla profit 3.0
 quantity 2.0
2018-04-02 Chocolate profit 3.0
 quantity 9.0
 Cookie Dough profit 6.0
 quantity 3.0
 Green Tea profit 3.0
 quantity 10.0
 Vanilla profit 1.5
 quantity 8.0
dtype: float64

With a “stacked” DataFrame, the inverse operation of stack is unstack, which by default unstacks the last level:

In [33]:
df_multi.stack()\
        .rename_axis(['date','flavor','type'],axis=0)\
        .unstack()\
        .head(8)

Out[ 72 ]: 
type profit quantity
date flavor 
2018-04-01 Chocolate 3.0 6.0
 Cookie Dough 1.5 6.0
 Green Tea 1.5 8.0
 Vanilla 3.0 2.0
2018-04-02 Chocolate 3.0 9.0
 Cookie Dough 6.0 3.0
 Green Tea 3.0 10.0
 Vanilla 1.5 8.0

The `stack()` or `unstack()` method can be applied to more than one level at a time by passing a list of levels. In the code cell below we pass a list of column names to the `unstack()` method.

In [35]:
df_multi.stack()\
        .rename_axis(['date','flavor','type'],axis=0)\
        .unstack(['flavor','type'])

Out[ 73 ]: 
flavor Chocolate Cookie Dough Green Tea \
type profit quantity profit quantity profit quantity 
date 
2018-04-01 3.0 6.0 1.5 6.0 1.5 8.0 
2018-04-02 3.0 9.0 6.0 3.0 3.0 10.0 
2018-04-03 3.0 10.0 3.0 5.0 6.0 2.0 
2018-04-04 3.0 6.0 3.0 3.0 7.5 8.0 
2018-04-05 1.5 1.0 6.0 5.0 4.5 1.0 
2018-04-06 7.5 1.0 7.5 8.0 7.5 7.0 

flavor Vanilla 
type profit quantity 
date 
2018-04-01 3.0 2.0 
2018-04-02 1.5 8.0 
2018-04-03 1.5 7.0 
2018-04-04 6.0 10.0 
2018-04-05 4.5 3.0 
2018-04-06 3.0 5.0

##3. Pivot tables

While `pivot()` provides general purpose pivoting with various data types (strings, numerics, etc.), pandas also provides `pivot_table()` for pivoting with aggregation of numeric data. The `pivot_table` method works like `pivot()`, but it aggregates the values from rows with duplicate entries for the specified columns.

Create a `pivot_table` instance of a DataFrame object:
- The `data` arguments takes a DataFrame object.
- The `index` and `columns` arguments take categorical variables which have duplicate values in the DataFrame. 
- The `values` arguments takes variable(s) that can be aggregated. 
- The `aggfunc` arguments takes the function to use for aggregation, defaulting to `numpy.mean`.

Create a pivot table from the `import_df` DataFrame. In the pivot table we calculate the average `horsepower` for each `body_style` according to the type of `drive_wheels`.

In [40]:
pd.pivot_table(data=import_df, index='body_style', columns='drive_wheels', values='horsepower', aggfunc=np.mean)

Out[ 74 ]: 
drive_wheels 4wd fwd rwd
body_style 
convertible NaN 90.000000 140.000000
hardtop NaN 69.000000 152.714286
hatchback 116.500000 84.791667 141.526316
sedan 102.666667 86.631579 129.222222
wagon 79.250000 88.909091 116.777778

Below, create a pivot table from the `import_df` DataFrame that calculates the average `price` for each `body_style`. The result object `res` is a Series. The values are rounded to zero decimals usingthe `np.round()` function.

In [42]:
res = pd.pivot_table(import_df, 
                     values ='price', 
                     index='body_style',
                     aggfunc=np.mean)
np.round(res)

Out[ 75 ]: 
body_style
convertible 21890.0
hardtop 22208.0
hatchback 9957.0
sedan 14460.0
wagon 12372.0
Name: price, dtype: float64

Create a pivot table from the `import_df` DataFrame. The result object `res` is a DataFrame having hierarchical indexes on the rows. Omit the missing values by calling `to_string` method.

In [44]:
res = pd.pivot_table(import_df, 
                     values ='price', 
                     columns='body_style', 
                     index  =['make','drive_wheels'], 
                     aggfunc=np.mean)\
        .round()

print(res.to_string(na_rep=''))

body_style convertible hardtop hatchback sedan wagon
make drive_wheels 
alfa-romero rwd 14998.0 16500.0 
audi 4wd 17450.0 
 fwd 17696.0 18920.0
bmw rwd 26119.0 
chevrolet fwd 5723.0 6575.0 
dodge fwd 7820.0 7620.0 8921.0
honda fwd 7054.0 9945.0 7295.0
isuzu fwd 
 rwd 11048.0 6785.0 
jaguar rwd 34600.0 
mazda fwd 8128.0 8725.0 
 rwd 13020.0 18312.0 
mercedes-benz rwd 35056.0 36788.0 33074.0 28248.0
mercury rwd 16503.0 
mitsubishi fwd 9598.0 8434.0 
nissan fwd 8249.0 8374.0 8605.0 9916.0
 rwd 18432.0 
peugot rwd 15759.0 15018.0
plymouth fwd 6586.0 7150.0 8921.0
 rwd 12764.0 
porsche rwd 37028.0 33278.0 22018.0 
renault fwd 9895.0 9295.0
saab fwd 15013.0 15433.0 
subaru 4wd 7603.0 10246.0 9854.0
 fwd 6086.0 8287.0 8830.0
toyota 4wd 8338.0
 fwd 7844.0 8911.0 6918.0
 rwd 17669.0 9762.0 11978.0 11015.0 15750.0
volkswagen fwd 11595.0 9980.0 9674.0 12290.0
volvo rwd 18727.0 16293.0

Note that `pivot_table` is also available as an instance method on DataFrame, i.e. `DataFrame.pivot_table()`.

Create a pivot table from `import_df` by calling the `pivot_table` method. 
Since the `values` column name is not given, after grouping by the `make` variable, all columns are aggregated using the `np.mean` function.

In [47]:
import_df.pivot_table(index='make').round()

Out[ 77 ]: 
 bore city_mpg compression_ratio curb_weight engine_size \
make 
alfa-romero 3.0 20.0 9.0 2640.0 137.0 
audi 3.0 19.0 8.0 2801.0 131.0 
bmw 3.0 19.0 9.0 2929.0 167.0 
chevrolet 3.0 41.0 10.0 1757.0 80.0 
dodge 3.0 28.0 9.0 2151.0 103.0 
honda 3.0 30.0 9.0 2097.0 99.0 
isuzu 3.0 31.0 9.0 2214.0 102.0 
jaguar 4.0 14.0 9.0 4027.0 281.0 
mazda 3.0 26.0 10.0 2298.0 103.0 
mercedes-benz 4.0 18.0 15.0 3696.0 226.0 
mercury 4.0 19.0 8.0 2910.0 140.0 
mitsubishi 3.0 25.0 8.0 2382.0 118.0 
nissan 3.0 27.0 10.0 2400.0 128.0 
peugot 4.0 22.0 14.0 3221.0 136.0 
plymouth 3.0 28.0 9.0 2221.0 106.0 
porsche 4.0 17.0 10.0 2891.0 187.0 
renault 3.0 23.0 9.0 2520.0 132.0 
saab 3.0 20.0 9.0 2746.0 121.0 
subaru 4.0 26.0 9.0 2316.0 107.0 
toyota 3.0 28.0 10.0 2441.0 119.0 
volkswagen 3.0 29.0 14.0 2343.0 107.0 
volvo 4.0 21.0 10.0 3038.0 142.0 

 height highway_mpg horsepower length normalized_losses \
make 
alfa-romero 50.0 27.0 125.0 170.0 NaN 
audi 54.0 24.0 121.0 184.0 161.0 
bmw 55.0 25.0 139.0 184.0 190.0 
chevrolet 52.0 46.0 63.0 152.0 100.0 
dodge 52.0 34.0 86.0 161.0 133.0 
honda 53.0 35.0 80.0 161.0 103.0 
isuzu 52.0 36.0 77.0 164.0 NaN 
jaguar 51.0 18.0 205.0 197.0 145.0 
mazda 53.0 32.0 86.0 171.0 124.0 
mercedes-benz 56.0 21.0 146.0 195.0 103.0 
mercury 55.0 24.0 175.0 178.0 NaN 
mitsubishi 51.0 31.0 104.0 168.0 146.0 
nissan 54.0 33.0 103.0 171.0 135.0 
peugot 57.0 27.0 100.0 191.0 161.0 
plymouth 52.0 34.0 87.0 165.0 129.0 
porsche 51.0 26.0 210.0 170.0 186.0 
renault 53.0 31.0 NaN 179.0 NaN 
saab 56.0 27.0 127.0 187.0 127.0 
subaru 54.0 31.0 86.0 169.0 92.0 
toyota 54.0 33.0 93.0 172.0 110.0 
volkswagen 55.0 35.0 81.0 173.0 121.0 
volvo 56.0 26.0 128.0 189.0 91.0 

 peak_rpm price stroke symboling wheel_base width 
make 
alfa-romero 5000.0 15498.0 3.0 2.0 91.0 65.0 
audi 5500.0 17859.0 3.0 1.0 102.0 69.0 
bmw 5069.0 26119.0 3.0 0.0 103.0 66.0 
chevrolet 5300.0 6007.0 3.0 1.0 92.0 62.0 
dodge 5389.0 7875.0 3.0 1.0 95.0 64.0 
honda 5754.0 8185.0 3.0 1.0 94.0 64.0 
isuzu 5150.0 8916.0 3.0 1.0 95.0 64.0 
jaguar 4833.0 34600.0 4.0 0.0 109.0 70.0 
mazda 5109.0 10653.0 3.0 1.0 97.0 66.0 
mercedes-benz 4488.0 33647.0 3.0 0.0 111.0 71.0 
mercury 5000.0 16503.0 3.0 1.0 103.0 68.0 
mitsubishi 5269.0 9240.0 3.0 2.0 95.0 65.0 
nissan 5178.0 10416.0 3.0 1.0 96.0 65.0 
peugot 4668.0 15489.0 3.0 0.0 110.0 68.0 
plymouth 5357.0 7963.0 3.0 1.0 95.0 64.0 
porsche 5790.0 31400.0 3.0 3.0 92.0 67.0 
renault NaN 9595.0 4.0 1.0 96.0 67.0 
saab 5333.0 15223.0 3.0 2.0 99.0 66.0 
subaru 4775.0 8541.0 3.0 0.0 96.0 65.0 
toyota 4859.0 9886.0 3.0 1.0 98.0 65.0 
volkswagen 5154.0 10078.0 3.0 2.0 98.0 66.0 
volvo 5291.0 18063.0 3.0 -1.0 106.0 68.0

Note that the following command cell has the same output as the above one.

In [49]:
pd.pivot_table(import_df, index='make').round()

Out[ 78 ]: 
 bore city_mpg compression_ratio curb_weight engine_size \
make 
alfa-romero 3.0 20.0 9.0 2640.0 137.0 
audi 3.0 19.0 8.0 2801.0 131.0 
bmw 3.0 19.0 9.0 2929.0 167.0 
chevrolet 3.0 41.0 10.0 1757.0 80.0 
dodge 3.0 28.0 9.0 2151.0 103.0 
honda 3.0 30.0 9.0 2097.0 99.0 
isuzu 3.0 31.0 9.0 2214.0 102.0 
jaguar 4.0 14.0 9.0 4027.0 281.0 
mazda 3.0 26.0 10.0 2298.0 103.0 
mercedes-benz 4.0 18.0 15.0 3696.0 226.0 
mercury 4.0 19.0 8.0 2910.0 140.0 
mitsubishi 3.0 25.0 8.0 2382.0 118.0 
nissan 3.0 27.0 10.0 2400.0 128.0 
peugot 4.0 22.0 14.0 3221.0 136.0 
plymouth 3.0 28.0 9.0 2221.0 106.0 
porsche 4.0 17.0 10.0 2891.0 187.0 
renault 3.0 23.0 9.0 2520.0 132.0 
saab 3.0 20.0 9.0 2746.0 121.0 
subaru 4.0 26.0 9.0 2316.0 107.0 
toyota 3.0 28.0 10.0 2441.0 119.0 
volkswagen 3.0 29.0 14.0 2343.0 107.0 
volvo 4.0 21.0 10.0 3038.0 142.0 

 height highway_mpg horsepower length normalized_losses \
make 
alfa-romero 50.0 27.0 125.0 170.0 NaN 
audi 54.0 24.0 121.0 184.0 161.0 
bmw 55.0 25.0 139.0 184.0 190.0 
chevrolet 52.0 46.0 63.0 152.0 100.0 
dodge 52.0 34.0 86.0 161.0 133.0 
honda 53.0 35.0 80.0 161.0 103.0 
isuzu 52.0 36.0 77.0 164.0 NaN 
jaguar 51.0 18.0 205.0 197.0 145.0 
mazda 53.0 32.0 86.0 171.0 124.0 
mercedes-benz 56.0 21.0 146.0 195.0 103.0 
mercury 55.0 24.0 175.0 178.0 NaN 
mitsubishi 51.0 31.0 104.0 168.0 146.0 
nissan 54.0 33.0 103.0 171.0 135.0 
peugot 57.0 27.0 100.0 191.0 161.0 
plymouth 52.0 34.0 87.0 165.0 129.0 
porsche 51.0 26.0 210.0 170.0 186.0 
renault 53.0 31.0 NaN 179.0 NaN 
saab 56.0 27.0 127.0 187.0 127.0 
subaru 54.0 31.0 86.0 169.0 92.0 
toyota 54.0 33.0 93.0 172.0 110.0 
volkswagen 55.0 35.0 81.0 173.0 121.0 
volvo 56.0 26.0 128.0 189.0 91.0 

 peak_rpm price stroke symboling wheel_base width 
make 
alfa-romero 5000.0 15498.0 3.0 2.0 91.0 65.0 
audi 5500.0 17859.0 3.0 1.0 102.0 69.0 
bmw 5069.0 26119.0 3.0 0.0 103.0 66.0 
chevrolet 5300.0 6007.0 3.0 1.0 92.0 62.0 
dodge 5389.0 7875.0 3.0 1.0 95.0 64.0 
honda 5754.0 8185.0 3.0 1.0 94.0 64.0 
isuzu 5150.0 8916.0 3.0 1.0 95.0 64.0 
jaguar 4833.0 34600.0 4.0 0.0 109.0 70.0 
mazda 5109.0 10653.0 3.0 1.0 97.0 66.0 
mercedes-benz 4488.0 33647.0 3.0 0.0 111.0 71.0 
mercury 5000.0 16503.0 3.0 1.0 103.0 68.0 
mitsubishi 5269.0 9240.0 3.0 2.0 95.0 65.0 
nissan 5178.0 10416.0 3.0 1.0 96.0 65.0 
peugot 4668.0 15489.0 3.0 0.0 110.0 68.0 
plymouth 5357.0 7963.0 3.0 1.0 95.0 64.0 
porsche 5790.0 31400.0 3.0 3.0 92.0 67.0 
renault NaN 9595.0 4.0 1.0 96.0 67.0 
saab 5333.0 15223.0 3.0 2.0 99.0 66.0 
subaru 4775.0 8541.0 3.0 0.0 96.0 65.0 
toyota 4859.0 9886.0 3.0 1.0 98.0 65.0 
volkswagen 5154.0 10078.0 3.0 2.0 98.0 66.0 
volvo 5291.0 18063.0 3.0 -1.0 106.0 68.0

__The End__